In [1]:
from transformers import AutoTokenizer,AutoModelForCausalLM
import torch

model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
Token = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)

c:\Users\User\miniconda3\envs\ai-core\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


使用TinyLlama做測試與學習

Token與model的Pretrained

函數庫:

AutoTokenizer  對應Token

AutoModelForCausalLM 對應自然語言LLM

In [2]:
print(Token)

LlamaTokenizerFast(name_or_path='TinyLlama/TinyLlama-1.1B-Chat-v1.0', vocab_size=32000, model_max_length=2048, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '</s>'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)


Tokenizer = 把文字切成 token ID 的工具

vocab_size=32000：字典大小，有 32k 個 token（詞彙單位）

model_max_length=2048：最多能處理 2048 個 token 的序列

special_tokens：像 BOS <s>、EOS </s>，控制輸入/輸出的邊界

這是「輸入管道」：文字先經過 tokenizer → 轉成數字 ID

In [3]:
print(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 2048)
    (layers): ModuleList(
      (0-21): 22 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (up_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (down_proj): Linear(in_features=5632, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((2048,), eps=1e-05)
    (rotary_emb): 

Embedding(32000, 2048)
每個 token ID 對應一個 2048 維的向量

22 層 LlamaDecoderLayer
這就是 Transformer Decoder 堆疊

每層有 Self-Attention（Q/K/V 投影 + 輸出）

MLP 前饋層（gate_proj → up_proj → down_proj）

LayerNorm（RMSNorm）

LlamaAttention 裡特別之處：
q_proj: 2048→2048
k_proj, v_proj: 2048→256（壓縮了維度 → 減少運算量）
o_proj: 2048→2048

RotaryEmbedding (RoPE)
用於 attention 的位置編碼，讓模型理解 token 順序

lm_head (Linear 2048→32000)
最後一層，全連接層，輸出每個 token 的機率分佈